In [ ]:
import sys
sys.path.append('..')

In [1]:
from utils import SocketReader

In [2]:
from new_parser import construct, match, match_exact

In [3]:
try:
    sock.close()
except:
    pass

sock = SocketReader("echo.websocket.org", 443, 'tls')

sock.connect()

In [4]:
message = b"""GET /.ws HTTP/1.1
Host: echo.websocket.org
Upgrade: websocket
Connection: Upgrade
Sec-WebSocket-Key: x3JJHMbDL1EzLkh9GBhXDw==
Sec-WebSocket-Version: 13
Origin: http://echo.websocket.org

"""
sock.send(message)

In [5]:
sock.read_line()

b'HTTP/1.1 101 Switching Protocols\r\n'

In [6]:
sock.read_line()

b'upgrade: websocket\r\n'

In [7]:
from base64 import b64encode
from hashlib import sha1

content_length = None
chunked = False

while True:
    header = sock.read_line()
    print(header)
    if header == b'\r\n':
        break
    if header.lower().startswith(b'sec-websocket-accept:'):
        result_hash = header.split(b':')[1].strip()
        key = b"x3JJHMbDL1EzLkh9GBhXDw==" # The key sent by the client
        magic = b"258EAFA5-E914-47DA-95CA-C5AB0DC85B11" # Protocol constant
        assert b64encode(sha1(key + magic).digest()) == result_hash, "the result hash doesn't match the expected hash"
        


b'connection: Upgrade\r\n'
b'sec-websocket-accept: HSmrc0sMlYUkAGmm5OPpG2HaGWk=\r\n'
b'date: Mon, 08 Jul 2024 15:26:01 GMT\r\n'
b'server: Fly/577378ac (2024-07-02)\r\n'
b'via: 1.1 fly.io\r\n'
b'fly-request-id: 01J29F7BN5YHP06QGVA74P9S11-ams\r\n'
b'\r\n'


In [8]:
# message_spec = """
# 1@fin
# -3@rsv
# -4@opcode
# -1@masked
# -(
#         7==127-64<@size
#         |7==126-16@size
#         |7@size
# )
# -{masked}...@masking_key(32)
# -{size}@payload
# """

# maskval = random.randrange(1<<32)

# message = bytearray(b"hello")

# for i in range(len(message)):
#     message[i] = message[i] ^ mask[i % 4]
    
# frame = {
#     "fin": 0b1,
#     "rsv": 0b000,
#     "opcode": 1, # text
#     "masking_key": [maskval],
#     "payload": message
# }



# print(construct(message_spec, frame))


# frame = {
#     "fin": 0b1,
#     "rsv": 0b000,
#     "opcode": 1, # text
#     "masked": 1,
#     "payload_length_prefix": len(message), # <= 125
# }

# import random
# mask = construct(mask_spec, {'mask': maskval})
# message
# frame_encoded = construct(frame_spec, frame) + mask + bytes(message)
# print(frame_encoded)

In [9]:
server_to_client_message_spec = """
1@fin
-3@rsv
-4@opcode
-1==0@masked
-(
        7==127-64@size
        |7==126-16@size
        |7@size
)
-{masked}...@masking_key(32)
-{size}@payload
"""

# TODO: fix 126, 127 message size
client_to_server_message_spec = """
1@fin
-3@rsv
-4@opcode
-1==1@masked
-(
        7@size
        |7==126-16@size
        |7==127-64@size
)
-{masked}...@masking_key(32)
-{size}@payload
"""


In [10]:
def reader():
    while True:
        message = sock.read_once()
        received_frame = match_exact(server_to_client_message_spec, message, verbose=True)
        if received_frame['size'] != 0:
            print(received_frame['payload']._value('bytes'))
import threading

threading.Thread(target=reader).start()

b'Request served by 1781505b56ee58'
b'hello'
b'p---ping'


Exception in thread Thread-5 (reader):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/home/mie/.virtualenvs/notebooks/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_2068270/2230967281.py", line 4, in reader
  File "/home/mie/notebooks/new_parser.py", line 689, in match_exact
    res, rem = match(expression, data, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mie/notebooks/new_parser.py", line 683, in match
    res, rem = deevaluate(parse_tree, binary_string, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mie/notebooks/new_parser.py", line 579, in deevaluate
    res, data = deevaluate(tree[1], data, curr=context, **kwargs)
                ^^^^^^^^

In [11]:
data = bytearray(b"hello")

In [12]:
import random
maskval = random.randrange(1<<32)
for i in range(len(data)):
    data[i] = data[i] ^ ((maskval >> (8*(3-(i%4)))) & 0xFF)
data

bytearray(b'n\xc6+\x8ei')

In [13]:
frame = {
    "fin": 0b1,
    "rsv": 0b000,
    "opcode": 1, # text
    "masking_key": [maskval],
    "payload": data
}

frame_encoded = construct(client_to_server_message_spec, frame)
frame_encoded

b'\x81\x85\x06\xa3G\xe2n\xc6+\x8ei'

In [14]:
sock.send(frame_encoded)

In [15]:
data = bytearray(b"p---ping")
maskval = random.randrange(1<<32)
for i in range(len(data)):
    data[i] = data[i] ^ ((maskval >> (8*(3-(i%4)))) & 0xFF)
data


bytearray(b'\xa1[^\x8b\xa1\x1f\x1d\xc1')

In [16]:
frame = {
    "fin": 0b1,
    "rsv": 0b000,
    "opcode": 9, # ping
    "masking_key": [maskval],
    "payload": data
}

frame_encoded = construct(client_to_server_message_spec, frame)
frame_encoded

b'\x89\x88\xd1vs\xa6\xa1[^\x8b\xa1\x1f\x1d\xc1'

In [17]:
sock.send(frame_encoded)

In [18]:
sock.close()